In [1]:
import os

import albumentations as A
import cv2
import numpy as np
import pandas as pd
import PIL
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import yaml
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from torch.cuda import amp
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

from models.tracker import YoloTracker
from utils.general import labels_to_class_weights
from utils.loss import ComputeLoss


def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

class MOT17Dataset(Dataset):
    def __init__(self, ds_path: str = r'K:\Users\krish\Downloads\MOT17\train', transforms = None, subset='train') -> None:
        super().__init__()
        self.base_path = ds_path
        gt_path = ds_path + r'\<seqname>\gt\gt.txt'
        self.img_path = ds_path + r'\<seqname>\img1'
        file_list = os.listdir(ds_path)
        self.transforms = transforms
        self.df = pd.DataFrame(columns=['seqname', 'frame', 'ID',
                                        'bb_left', 'bb_top', 'bb_width', 'bb_height', 'x', 'y', 'z'])
        
        for file in file_list:
            df = pd.read_csv(gt_path.replace('<seqname>', file), names=[
                'frame', 'ID', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'x', 'y', 'z'])
            df['seqname'] = file
            df = df.sort_values('frame')
            self.df = pd.concat([self.df, df])
        self.df = self.df.reset_index(drop=True)
        self.df['centery'] = (self.df['bb_top'] + self.df['bb_height'])/2
        self.df['centerx'] = (self.df['bb_left'] + self.df['bb_width'])/2
        tr, ts = train_test_split(self.df, train_size=0.8, random_state=42)
        if subset == 'train':
            self.df = tr
        elif subset == 'test':
            self.df = ts
        else:
            raise NameError
    
    def __len__(self):
        return self.df.seqname.nunique()

    def __get_item__(self, idx, img_size = 640):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        seqname = self.df.seqname.unique()[idx]
        seqdata = self.df[self.df.seqname==seqname]
        
        #load images (frames)
        rescalers = []
        samples = []
        for img_file in os.listdir(self.img_path.replace('<seqname>', seqname)):
            img = load_image(os.path.join(self.img_path.replace('<seqname>', seqname), img_file))
            h, w, c = img.shape
            resized = cv2.resize(img, (img_size, img_size), interpolation = cv2.INTER_AREA)
            rescalers.append([resized.shape[0]/h, resized.shape[1]/w]) #multiply the coordinates with these to obtain rescaled values
            samples.append(resized)

        y_cols = ['bb_top', 'bb_height', 'centery']
        x_cols = ['bb_left', 'bb_width', 'centerx']
        
        for col in y_cols:
            seqdata[col] = seqdata[col].apply(lambda x: x*rescalers[0][0]/h)
        for col in x_cols:
            seqdata[col] = seqdata[col].apply(lambda x: x*rescalers[0][1]/w)
        
        
        if self.transforms is not None:
            samples = [self.transforms(image=img)['image'] for img in samples]
        
        #label
        seq_heatmaps = []
        seq_bboxes = []
        seq_ids = []
        for frame_id in seqdata.frame.unique():
            frame_data = seqdata[seqdata.frame == frame_id]
            #parse labels
            bboxes = []
            ids = []
            ht_mp = torch.zeros((img_size, img_size))
            for i, elem in frame_data.iterrows():
                
                ht_mp[int(elem.centery)][int(elem.centerx)] = 1
                #list of bounding_boxes
                bboxes.append([0, elem.centerx, elem.centery, elem.bb_width, elem.bb_height])
                ids.append(elem.ID)
            seq_heatmaps.append(ht_mp)
            seq_bboxes.append(bboxes)
            seq_ids.append(ids)
        
        assert len(seq_heatmaps) == len(samples)
        
        return samples, seq_bboxes, seq_ids, seq_heatmaps

def train(ds_path = r'D:\datasets\MOT17\MOT17\train', batch_size=1, 
                cfg = './cfg/training/yolov7.yaml',
                hyp = './data/hyp.scratch.p5.yaml',
                nc = 80,
                epochs=30,
                device='cuda'):

    with open(hyp) as f:
        hyp = yaml.load(f, Loader=yaml.SafeLoader)  # load hyps
    
    transform = A.Compose([
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
        ])
    train_dataset = MOT17Dataset(ds_path=ds_path, transforms=transform, subset='train')
    test_dataset = MOT17Dataset(ds_path=ds_path, transforms=transform, subset='test')


c:\Users\luido\Anaconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds_path = r'D:\datasets\MOT17\MOT17\train'
batch_size=1 
cfg = './cfg/training/yolov7.yaml'
hyp = './data/hyp.scratch.p5.yaml'
nc = 80
epochs=30
device='cuda'

transform = A.Compose([
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
        ])
train_dataset = MOT17Dataset(ds_path=ds_path, transforms=transform, subset='train')
test_dataset = MOT17Dataset(ds_path=ds_path, transforms=transform, subset='test')
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True,)
# test_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True,)
# nb = len(train_loader)

In [3]:
seqs = []
for i in range(len(train_dataset)):
    seqs.append(train_dataset.__get_item__(i))
    break

Chegou


C:\Users\luido\AppData\Local\Temp\ipykernel_21592\3395472585.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seqdata[col] = seqdata[col].apply(lambda x: x*rescalers[0][0]/h)
C:\Users\luido\AppData\Local\Temp\ipykernel_21592\3395472585.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seqdata[col] = seqdata[col].apply(lambda x: x*rescalers[0][1]/w)


In [21]:
len(seqs[0][0])

654

In [23]:
seqs[0][1]

[[[0, 214.5, 160.88888888888889, 23.333333333333332, 153.48148148148147],
  [0, 105.5, 124.14814814814814, 12.0, 49.18518518518518],
  [0,
   356.66666666666663,
   173.92592592592592,
   186.33333333333331,
   142.8148148148148],
  [0, 145.16666666666666, 143.4074074074074, 20.0, 97.18518518518518],
  [0, 118.16666666666666, 120.59259259259258, 8.0, 38.51851851851852],
  [0, 174.16666666666666, 114.96296296296296, 7.0, 24.296296296296294],
  [0, 210.0, 157.33333333333331, 22.0, 136.29629629629628],
  [0,
   256.3333333333333,
   125.03703703703702,
   14.333333333333332,
   74.07407407407408],
  [0, 187.33333333333331, 121.77777777777777, 11.0, 47.407407407407405],
  [0,
   87.66666666666666,
   215.1111111111111,
   72.33333333333333,
   284.44444444444446],
  [0,
   200.16666666666666,
   138.96296296296296,
   20.333333333333332,
   88.88888888888889],
  [0,
   167.33333333333331,
   119.1111111111111,
   8.666666666666666,
   39.11111111111111],
  [0, 82.66666666666666, 152.0, 19.